
# **Deep Learning - CSE4006**
# LAB 4

> ## Name: **K CHARVI**
> ## Reg No: 19BCE7002
> ## Date: 25|10|2021
> ## Submitted To: Dr. BKSP Kumarraju Alluri


# Project 2: Road Lane Detection


In [ ]:
!git clone "https://github.com/mvirgo/MLND-Capstone"

Cloning into 'MLND-Capstone'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 301 (delta 1), reused 3 (delta 0), pack-reused 293
Receiving objects: 100% (301/301), 26.06 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [1]:
%cd /content/drive/MyDrive/Deep learning LAB

/content/drive/MyDrive/Deep learning LAB


In [2]:
# Load training images
import pickle
train_images = pickle.load(open("full_CNN_train.p", "rb" ))

In [3]:
# Load image labels
labels = pickle.load(open("full_CNN_labels.p", "rb"))

In [4]:
# Make into arrays as the neural network wants these
import numpy as np
train_images = np.array(train_images)
labels = np.array(labels)

In [5]:
# Normalize labels - training images get normalized to start in the network
labels = labels / 255

In [6]:
# Shuffle images along with their labels, then split into training/validation sets
from sklearn.utils import shuffle
train_images, labels = shuffle(train_images, labels)

In [7]:
# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from tensorflow.keras.layers import LayerNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers


In [8]:
def create_model(input_shape, pool_size):
    # Create the actual neural network here
    model = Sequential()
    # Normalizes incoming inputs. First layer needs the input shape to work
    model.add(LayerNormalization(input_shape=input_shape))

    # Below layers were re-named for easier reading of model summary; this not necessary
    # Conv Layer 1
    model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'))

    # Conv Layer 2
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))

    # Pooling 1
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 3
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
    model.add(Dropout(0.2))

    # Conv Layer 4
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
    model.add(Dropout(0.2))

    # Conv Layer 5
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
    model.add(Dropout(0.2))

    # Pooling 2
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 6
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'))
    model.add(Dropout(0.2))

    # Conv Layer 7
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'))
    model.add(Dropout(0.2))

    # Pooling 3
    model.add(MaxPooling2D(pool_size=pool_size))

    # Upsample 1
    model.add(UpSampling2D(size=pool_size))

    # Deconv 1
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'))
    model.add(Dropout(0.2))

    # Deconv 2
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))
    model.add(Dropout(0.2))

    # Upsample 2
    model.add(UpSampling2D(size=pool_size))

    # Deconv 3
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'))
    model.add(Dropout(0.2))

    # Deconv 4
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'))
    model.add(Dropout(0.2))

    # Deconv 5
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'))
    model.add(Dropout(0.2))

    # Upsample 3
    model.add(UpSampling2D(size=pool_size))

    # Deconv 6
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'))

    # Final layer - only including one channel so 1 filter
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Final'))

    return model


In [9]:
# Test size may be 10% or 20%
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

In [10]:
# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 10
pool_size = (2, 2)
input_shape = X_train.shape[1:]


In [11]:
# Create the neural network
model = create_model(input_shape, pool_size)

In [12]:
# Using a generator to help the model use less data
# Channel shifts help with shadows slightly
datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)

In [13]:
# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'])
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
epochs=epochs, verbose=1, validation_data=(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
89/89 [==============================] - 83s 544ms/step - loss: 0.0676 - accuracy: 0.8866 - val_loss: 0.0694 - val_accuracy: 0.8572
Epoch 2/10
89/89 [==============================] - 44s 492ms/step - loss: 0.0288 - accuracy: 0.9360 - val_loss: 0.0286 - val_accuracy: 0.9395
Epoch 3/10
89/89 [==============================] - 44s 490ms/step - loss: 0.0218 - accuracy: 0.9436 - val_loss: 0.0291 - val_accuracy: 0.9363
Epoch 4/10
89/89 [==============================] - 44s 489ms/step - loss: 0.0189 - accuracy: 0.9466 - val_loss: 0.0192 - val_accuracy: 0.9461
Epoch 5/10
89/89 [==============================] - 44s 489ms/step - loss: 0.0168 - accuracy: 0.9488 - val_loss: 0.0217 - val_accuracy: 0.9438
Epoch 6/10
89/89 [==============================] - 44s 489ms/step - loss: 0.0158 - accuracy: 0.9497 - val_loss: 0.0172 - val_accuracy: 0.9479
Epoch 7/10
89/89 [==============================] - 44s 488ms/step - loss: 0.0150 - accuracy: 0.9505 - val_loss: 0.0166 - val_accuracy: 0.9484

In [14]:
# Freeze layers since training is done
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'])

In [15]:
# Show summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization (LayerNo (None, 80, 160, 3)        6         
_________________________________________________________________
Conv1 (Conv2D)               (None, 78, 158, 8)        224       
_________________________________________________________________
Conv2 (Conv2D)               (None, 76, 156, 16)       1168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 78, 16)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 36, 76, 16)        2320      
_________________________________________________________________
dropout (Dropout)            (None, 36, 76, 16)        0         
_________________________________________________________________
Conv4 (Conv2D)               (None, 34, 74, 32)        4

In [16]:
# Save model architecture and weights
model.save('full_CNN_model.h5')